# ⚽ PID Controller Ball Chase

The red ball uses predictive steering to chase the blue ball moving in a circle.

This implementation demonstrates how simple predictive control can effectively track a moving target!

## Interactive HTML5 Canvas Animation

Run this cell to see the animation with full JavaScript controls!

In [ ]:
from IPython.display import HTML

html_code = '''
<!DOCTYPE html>
<html>
<head>
    <style>
        body {
            background-color: #1a1a2e;
            margin: 0;
            padding: 20px;
            font-family: Arial, sans-serif;
        }
        #canvas-container {
            text-align: center;
        }
        canvas {
            background-color: #1a1a2e;
            border: 2px solid #e74c3c;
            border-radius: 10px;
            cursor: pointer;
        }
        .controls {
            margin: 20px auto;
            max-width: 700px;
            background: #2a2a3e;
            padding: 20px;
            border-radius: 10px;
        }
        button {
            background: #e74c3c;
            color: white;
            border: none;
            padding: 10px 20px;
            margin: 5px;
            border-radius: 5px;
            cursor: pointer;
            font-size: 14px;
        }
        button:hover { background: #c0392b; }
        .slider-container {
            margin: 15px 0;
            color: white;
        }
        input[type="range"] {
            width: 100%;
            margin: 10px 0;
        }
        label {
            display: block;
            margin-bottom: 5px;
            font-weight: bold;
        }
    </style>
</head>
<body>
    <div id="canvas-container">
        <canvas id="canvas" width="700" height="700"></canvas>
    </div>
    <div class="controls">
        <div>
            <button onclick="resetSimulation()">🔄 Reset</button>
            <button onclick="togglePause()">⏸️ Pause</button>
        </div>
        <div class="slider-container">
            <label>Circle Radius: <span id="radius-value">120</span> px</label>
            <input type="range" id="radius-slider" min="50" max="200" step="10" value="120" 
                   oninput="updateRadius(this.value)">
        </div>
    </div>

    <script>
        const canvas = document.getElementById('canvas');
        const ctx = canvas.getContext('2d');
        
        let paused = false;
        
        // Simulation State
        let sim = {
            center: {x: 350, y: 350},
            radius: 120,
            blueAngle: 0,
            blueSpeed: 0.008,
            blueRadius: 12,
            redPos: {x: 350 - 120, y: 350},
            redRadius: 12,
            blueTrail: [],
            redTrail: [],
            maxTrail: 200,
            caught: false,
            catchDistance: 25
        };
        
        sim.redSpeed = sim.radius * sim.blueSpeed;
        
        function resetSimulation() {
            sim.blueAngle = 0;
            sim.redPos = {x: sim.center.x - sim.radius, y: sim.center.y};
            sim.blueTrail = [];
            sim.redTrail = [];
            sim.caught = false;
            sim.redSpeed = sim.radius * sim.blueSpeed;
        }
        
        function togglePause() {
            paused = !paused;
            document.querySelector('button[onclick="togglePause()"]').textContent = 
                paused ? '▶️ Resume' : '⏸️ Pause';
        }
        
        function updateRadius(value) {
            sim.radius = parseFloat(value);
            document.getElementById('radius-value').textContent = value;
            resetSimulation();
        }
        
        function update() {
            if (paused) return;
            
            // Update blue ball
            sim.blueAngle += sim.blueSpeed;
            const bluePos = {
                x: sim.center.x + sim.radius * Math.cos(sim.blueAngle),
                y: sim.center.y + sim.radius * Math.sin(sim.blueAngle)
            };
            
            // Predict where blue ball will be (several frames ahead)
            const predictFrames = 10;
            const futureAngle = sim.blueAngle + sim.blueSpeed * predictFrames;
            const futureBluePos = {
                x: sim.center.x + sim.radius * Math.cos(futureAngle),
                y: sim.center.y + sim.radius * Math.sin(futureAngle)
            };
            
            // Red ball moves toward predicted position
            const dx = futureBluePos.x - sim.redPos.x;
            const dy = futureBluePos.y - sim.redPos.y;
            const dist = Math.sqrt(dx*dx + dy*dy);
            
            if (dist > 0) {
                sim.redPos.x += (dx/dist) * sim.redSpeed;
                sim.redPos.y += (dy/dist) * sim.redSpeed;
            }
            
            // Check caught
            const catchDist = Math.sqrt(
                Math.pow(bluePos.x - sim.redPos.x, 2) + 
                Math.pow(bluePos.y - sim.redPos.y, 2)
            );
            sim.caught = catchDist < sim.catchDistance;
            
            // Update trails
            sim.blueTrail.push({...bluePos});
            sim.redTrail.push({...sim.redPos});
            
            if (sim.blueTrail.length > sim.maxTrail) {
                sim.blueTrail.shift();
                sim.redTrail.shift();
            }
            
            return {bluePos, catchDist};
        }
        
        function draw() {
            const {bluePos, catchDist} = update() || {};
            
            // Clear canvas
            ctx.fillStyle = '#1a1a2e';
            ctx.fillRect(0, 0, canvas.width, canvas.height);
            
            // Draw circular path
            ctx.strokeStyle = 'rgba(255, 255, 255, 0.3)';
            ctx.lineWidth = 2;
            ctx.setLineDash([5, 5]);
            ctx.beginPath();
            ctx.arc(sim.center.x, sim.center.y, sim.radius, 0, Math.PI * 2);
            ctx.stroke();
            ctx.setLineDash([]);
            
            // Draw trails
            if (sim.blueTrail.length > 1) {
                ctx.strokeStyle = 'rgba(52, 152, 219, 0.3)';
                ctx.lineWidth = 1.5;
                ctx.beginPath();
                ctx.moveTo(sim.blueTrail[0].x, sim.blueTrail[0].y);
                for(let i = 1; i < sim.blueTrail.length; i++) {
                    ctx.lineTo(sim.blueTrail[i].x, sim.blueTrail[i].y);
                }
                ctx.stroke();
                
                ctx.strokeStyle = 'rgba(231, 76, 60, 0.3)';
                ctx.beginPath();
                ctx.moveTo(sim.redTrail[0].x, sim.redTrail[0].y);
                for(let i = 1; i < sim.redTrail.length; i++) {
                    ctx.lineTo(sim.redTrail[i].x, sim.redTrail[i].y);
                }
                ctx.stroke();
            }
            
            // Draw balls
            if (bluePos) {
                ctx.fillStyle = '#3498db';
                ctx.beginPath();
                ctx.arc(bluePos.x, bluePos.y, sim.blueRadius, 0, Math.PI * 2);
                ctx.fill();
            }
            
            ctx.fillStyle = '#e74c3c';
            ctx.beginPath();
            ctx.arc(sim.redPos.x, sim.redPos.y, sim.redRadius, 0, Math.PI * 2);
            ctx.fill();
            
            // Draw status
            const statusText = sim.caught ? '🎉 Caught! ✨' : '🎯 Chasing...';
            const statusColor = sim.caught ? '#ff9800' : '#e74c3c';
            
            ctx.fillStyle = statusColor;
            ctx.fillRect(260, 630, 180, 40);
            ctx.fillStyle = 'white';
            ctx.font = 'bold 16px Arial';
            ctx.textAlign = 'center';
            ctx.fillText(statusText, 350, 655);
            
            // Draw info
            if (catchDist !== undefined) {
                ctx.fillStyle = 'white';
                ctx.font = '10px monospace';
                ctx.fillText(`Distance: ${catchDist.toFixed(1)} px`, 350, 30);
            }
            
            requestAnimationFrame(draw);
        }
        
        // Start animation
        draw();
    </script>
</body>
</html>
'''

HTML(html_code)

## 📐 PID Controller: Mathematical Foundation

### The Complete PID Control Law

$$u(t) = K_p \cdot e(t) + K_i \cdot \int_0^t e(\tau) d\tau + K_d \cdot \frac{de(t)}{dt}$$

Where:
- $u(t)$ = control output (what we do)
- $e(t) = \text{setpoint} - \text{measured value}$ = error
- $K_p$ = proportional gain
- $K_i$ = integral gain  
- $K_d$ = derivative gain

### The Three Components

#### 1. Proportional Term (P)
$$u_P(t) = K_p \cdot e(t)$$
- Immediate response to current error
- Larger error = larger response

#### 2. Integral Term (I)
$$u_I(t) = K_i \cdot \int_0^t e(\tau) d\tau$$
- Accumulates error over time
- Eliminates steady-state error

#### 3. Derivative Term (D)
$$u_D(t) = K_d \cdot \frac{de(t)}{dt}$$
- Responds to rate of change
- Reduces overshoot and oscillation